In [6]:
import pandas as pd
import psycopg2

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    dbname="moviedb",
    user="ajinkyaambadkar",
    password="Achiever216"  # Replace with your real PostgreSQL password
)

# Load ratings data
ratings_df = pd.read_sql("SELECT user_id, movie_id, rating FROM ratings;", conn)
conn.close()

ratings_df.head()
ratings_df.to_csv("data/processed/ratings_clean.csv", index=False)


/var/folders/z9/k5mpzs5n7yvdr65g96bpt72m0000gn/T/ipykernel_21681/4208180586.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings_df = pd.read_sql("SELECT user_id, movie_id, rating FROM ratings;", conn)


In [7]:
from surprise import Dataset, Reader

# Define a reader with rating scale 1–5
reader = Reader(rating_scale=(1, 5))

# Load ratings dataframe into Surprise dataset format
data = Dataset.load_from_df(ratings_df[['user_id', 'movie_id', 'rating']], reader)


In [8]:
from surprise import SVD
from surprise.model_selection import cross_validate

# Use SVD (matrix factorization)
model = SVD()

# Evaluate using 5-fold cross-validation
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9315  0.9338  0.9454  0.9368  0.9373  0.9370  0.0047  
MAE (testset)     0.7355  0.7357  0.7467  0.7372  0.7385  0.7387  0.0041  
Fit time          0.31    0.30    0.29    0.26    0.34    0.30    0.02    
Test time         0.03    0.06    0.03    0.06    0.03    0.04    0.01    


{'test_rmse': array([0.93151519, 0.93375345, 0.94544162, 0.93679859, 0.93727641]),
 'test_mae': array([0.73545069, 0.73574924, 0.74667344, 0.73715854, 0.73854165]),
 'fit_time': (0.3070828914642334,
  0.30084991455078125,
  0.2922041416168213,
  0.26270580291748047,
  0.3379089832305908),
 'test_time': (0.029619932174682617,
  0.05709123611450195,
  0.028538942337036133,
  0.05878186225891113,
  0.030364274978637695)}

In [9]:
trainset = data.build_full_trainset()
model.fit(trainset)


In [10]:
user_id = 100

# Get all movie IDs
all_movie_ids = ratings_df['movie_id'].unique()

# Get movies already rated by the user
rated_movies = ratings_df[ratings_df['user_id'] == user_id]['movie_id'].tolist()

# Get the ones they haven’t rated yet
unrated_movies = [mid for mid in all_movie_ids if mid not in rated_movies]

# Predict for all unrated movies
from surprise import Prediction
predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movies]

# Sort by estimated rating
top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

for pred in top_n:
    print(f"Movie ID: {pred.iid}, Predicted Rating: {pred.est:.2f}")


Movie ID: 318, Predicted Rating: 4.55
Movie ID: 496, Predicted Rating: 4.26
Movie ID: 12, Predicted Rating: 4.24
Movie ID: 64, Predicted Rating: 4.22
Movie ID: 427, Predicted Rating: 4.20
Movie ID: 83, Predicted Rating: 4.19
Movie ID: 169, Predicted Rating: 4.16
Movie ID: 114, Predicted Rating: 4.15
Movie ID: 515, Predicted Rating: 4.08
Movie ID: 143, Predicted Rating: 4.07


In [11]:
conn = psycopg2.connect(
    host="localhost",
    dbname="moviedb",
    user="ajinkyaambadkar",
    password="Achiever216"
)

movies_df = pd.read_sql("SELECT movie_id, title FROM movies;", conn)
conn.close()

movie_id_to_title = dict(zip(movies_df.movie_id, movies_df.title))
movies_df.to_csv("data/processed/movies_clean.csv", index=False)
# Show recommendations with titles
for pred in top_n:
    title = movie_id_to_title.get(int(pred.iid), "Unknown Movie")
    print(f"{title} - Predicted Rating: {pred.est:.2f}")


Schindler's List (1993) - Predicted Rating: 4.55
It's a Wonderful Life (1946) - Predicted Rating: 4.26
Usual Suspects, The (1995) - Predicted Rating: 4.24
Shawshank Redemption, The (1994) - Predicted Rating: 4.22
To Kill a Mockingbird (1962) - Predicted Rating: 4.20
Much Ado About Nothing (1993) - Predicted Rating: 4.19
Wrong Trousers, The (1993) - Predicted Rating: 4.16
Wallace & Gromit: The Best of Aardman Animation (1996) - Predicted Rating: 4.15
Boot, Das (1981) - Predicted Rating: 4.08
Sound of Music, The (1965) - Predicted Rating: 4.07


/var/folders/z9/k5mpzs5n7yvdr65g96bpt72m0000gn/T/ipykernel_21681/1391108185.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies_df = pd.read_sql("SELECT movie_id, title FROM movies;", conn)
